In [2]:
! pip install -U scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/f6/28/b569523552a11b49dc4d33952f43dedb23792fe8ce2f2151d070d615861a/scikit_learn-1.4.0-1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/f3/31/91a2a3c5eb85d2bfa86d7c98f2df5d77dcdefb3d80ca9f9037ad04393acf/scipy-1.12.0-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.4 kB 435.7 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/60.4 kB 435.7 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.4 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 321.8 kB/s eta 0:00:00
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/10/40


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import sklearn
import math

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB

from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import classification_report, accuracy_score

### Обработка датасета

In [2]:
# Считывание csv-файлов
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
gender_submission = pd.read_csv('gender_submission.csv')

# Объеденение датасетов
test_merge = pd.merge(test, gender_submission, how='left', on='PassengerId')
df = pd.concat([test_merge, train], ignore_index=True)

# Обработка Pclass
df['Pclass'] = df['Pclass'].apply(lambda x: 3 - x + 1)

# Обработка Sex
df['sex_rank'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1)

# Обработка Cabin
df['Cabin_class'] = df['Cabin'].apply(lambda x: np.nan if type(x) != str and np.isnan(x) else x[0])
df.iloc[[479, 1247], 10] = 'S'

# Обработка Embarked
df['embarked_s'] = df['Embarked'].apply(lambda x: 1 if x == 'S' else 0)
df['embarked_c'] = df['Embarked'].apply(lambda x: 1 if x == 'C' else 0)
df['embarked_q'] = df['Embarked'].apply(lambda x: 1 if x == 'Q' else 0)

# Итоговый датасет
df = df[['Pclass', 'sex_rank', 'Age', 'SibSp', 'Parch', 'Fare', 'embarked_s', 'embarked_c', 'embarked_q', 'Survived']]

# Обработка пропусков в Age и Fare 
age_median = df['Age'].median()
df['Age'] = df['Age'].apply(lambda x: age_median if np.isnan(x) else x)

# Удаление дублей
df.drop_duplicates(inplace=True, ignore_index=True)
df.iloc[145, 5] = df[df['Pclass'] == 1]['Fare'].mean()

### Краткий гайд по методу

Данная модель обучается методом подсчета вероятности каждого признака по формуле Байеса, при условии, что каждый признак **независим**!

In [52]:
%%html
<img src='images/slide-26.jpg', with=250, height=250>

**Пример расчета** 

https://www.youtube.com/watch?v=5x1EQ7Bpb5Q&list=LL&index=14&t=1s

#### Практический пример для нашего случая

Разберем подсчет вероятности к тому или иному классу, на примере одного столбца нашего датасета. Подсчитаем вероятность для события, что пассажир выживет, при условии, что он едет в 3 классе

In [56]:
# One feature dataset
ofd = df[['Pclass', 'Survived']]
ofd

,Pclass,Survived
0,1,0
1,1,1
2,2,0
3,1,0
4,1,1
...,...,...
1095,1,0
1096,3,1
1097,1,0
1098,3,1


**A** - Пассажир выжил(событие)

**B** - Класс пассажира равен **3** (событие)

**P(A)** - Вероятность того, что пассажир выжил

**P(B)** - Вероятность того, что класс пассажира = 3

**P(A|B)** - Вероятность того, что пассажир выжил, при условии что он ехал в 3 классе(Событие B произошло)

**P(B|A)** - Вероятность того, что пассажир ехал в 3 классе и выжил

In [60]:
p_A = len(ofd[ofd['Survived']==1])/len(ofd) # кол-во выжившых/кол-во всего

p_B = len(ofd[ofd['Pclass']==3])/len(ofd) # кол-во пассажиров 1 класса / кол-во всех пассажиров всех классов

p_BA = len(ofd[(ofd['Pclass']==3) & (ofd['Survived']==1)])/len(ofd[ofd['Pclass']==3]) # Кол-во выжившых пассажиров в классе 1/Кол-во пассажиров класса 1

In [61]:
p_AB = (p_A * p_BA) / p_B
p_AB

0.8627702360397432

Соответственно, для того чтобы отнести к тому или иному классу, нам нужно посчитать вероятность принадлежности ко всем этим классам (в нашем случае: выжил-1, погиб-0). Относим запись к тому классу, у которого **наибольшая** вероятность.

В случае, если имеется несколько признаков, сначала суммируем полученные вероятности посчитанные для каждого признака, а после классифицируем объект (по наибольшей сумме вероятностей)

Такой подход работает в случае, если мы считаем что признаки независимы (**наивный Байес**)

#### Наивный Байес для непрерывных переменных (**Гаусовский**)

Для подсчета вероятностей для признаков, которые имеются в качестве ннепрерывных переменных, считаем вероятность с использованием мат. ожидания(среднего) и дисперсии, по следующей формуле:

![fewf](images/ГаусовскийБК.png)

В случае если у нас несколько таких столбцов можно воспользоваться формулой (чтобы не усреднять):

![fewf](images/ГаусовскийБК_для_нескольких_переменных.png)

**Важно**
Этот будет работать плохо, если значения признаков **не подчиняются** Гаусовскому закону распределения

### Naive Bayess in Sklearn

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], train_size=0.80,
                                                    test_size=0.20, stratify=df.iloc[:, -1], random_state=123)

#### GaussianNB

Представляет собой классификатор, основанный на предположении, что вероятность наличия признаков равна гауссовскому распределению

In [10]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

sklearn.metrics.accuracy_score(y_pred, y_test)
#.score(y_pred, y_test)

0.7681818181818182

#### MultinomialNB

Он представляет собой классификатор, подходящий для многомерно распределенных данных.
Он хорошо подходит для задач классификации текста.

In [13]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

y_pred = mnb.predict(X_test)

sklearn.metrics.accuracy_score(y_pred, y_test)

0.7227272727272728

#### BernoulliNB

Он представляет собой классификатор, основанный на данных, представляющих собой многомерные распределения Бернулли.
Распределение Бернулли подразумевает, что данные могут иметь множество характеристик, но предполагается, что каждая из них является двоичной переменной.

In [14]:
bnb = BernoulliNB()
bnb.fit(X_train, y_train)

y_pred = bnb.predict(X_test)

sklearn.metrics.accuracy_score(y_pred, y_test)

0.8136363636363636

#### Усредненный accuracy используя все типы классификаторов (GaussianNB, MultinomialNB, BernoulliNB)

In [15]:
# Считаем вероятности для всех типов переменных (непрерывные, категориальные, бинарные)
gnb1 = GaussianNB()
gnb1.fit(X_train[['Age', 'Fare']], y_train)
y_pred_gnb1 = gnb1.predict_proba(X_test[['Age', 'Fare']])
y_pred_gnb1

mnb1 = MultinomialNB()
mnb1.fit(X_train[['Pclass', 'SibSp', 'Parch']], y_train)
y_pred_mnb1 = mnb1.predict_proba(X_test[['Pclass', 'SibSp', 'Parch']])
y_pred_mnb1

bnb1 = BernoulliNB()
bnb1.fit(X_train[['sex_rank', 'embarked_s', 'embarked_c', 'embarked_q']], y_train)
y_pred_bnb1 = bnb1.predict_proba(X_test[['sex_rank', 'embarked_s', 'embarked_c', 'embarked_q']])
y_pred_bnb1

# Усредняем полученные результаты
arr = np.array([])
for i in range(len(y_pred_bnb1)):
    probability_true = (y_pred_gnb1[i][0] + y_pred_mnb1[i][0] + y_pred_bnb1[i][0])/3
    probability_false = (y_pred_gnb1[i][1] + y_pred_mnb1[i][1] + y_pred_bnb1[i][1])/3

    arr = np.append(arr, 1 if probability_true>probability_false else 0)


sklearn.metrics.accuracy_score(arr, y_test)

0.20454545454545456

### Свой наивный Баейсовский классификатор Гауса, для бинарной классификации

In [71]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], train_size=0.80,
                                                    test_size=0.20, stratify=df.iloc[:, -1], random_state=123)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 0:-1], df.iloc[:, -1], train_size=0.8, test_size=0.2, random_state=42)

In [15]:
class СlassificationNB:
    def fit(self, x_train, y_train, category_columns:list=None, continuous_columns:list=None):
        self.count_feture_survived = {}
        self.count_feture_dead = {}

        for i in x_train.columns:
            survived_df, dead_df = x_train[y_train==1][i], x_train[y_train==0][i]
            m_survived, m_dead = survived_df.mean(), dead_df.mean()
            p_survived, p_dead = len(y_train[y_train==1])/len(y_train), len(y_train[y_train==0])/len(y_train)

            dispersion_survived = survived_df.apply(lambda x: (x-m_survived)**2).sum()/len(survived_df-1)
            dispersion_dead = dead_df.apply(lambda x: (x-m_dead)**2).sum()/len(dead_df-1)

            self.count_feture_survived[i] = [p_survived, m_survived, dispersion_survived]
            self.count_feture_dead[i] = [p_dead, m_dead, dispersion_dead]

        return (self.count_feture_survived, self.count_feture_dead)


    def predict(self, x_test):

        x_test = x_test.reset_index().iloc[:, 1:]

        total_prob_survived_df = pd.DataFrame({'surv': [0]*len(x_test)})
        total_prob_dead_df = pd.DataFrame({'dead': [0]*len(x_test)})

        for i in x_test.columns:

            total_prob_survived_df = pd.concat([total_prob_survived_df,
                x_test[i].apply(lambda x: math.log(self.count_feture_survived[i][0]) + (-((x-self.count_feture_survived[i][1])**2/(2*self.count_feture_survived[i][2]**2))))
                ], axis=1).sum(axis=1)

            total_prob_dead_df = pd.concat([total_prob_dead_df,
                x_test[i].apply(lambda x: math.log(self.count_feture_dead[i][0]) + (-((x-self.count_feture_dead[i][1])**2/(2*self.count_feture_dead[i][2]**2))))
                ], axis=1).sum(axis=1)

        print(total_prob_survived_df, total_prob_dead_df)

        return (total_prob_survived_df > total_prob_dead_df.values).map(lambda x: int(x))

In [18]:
my_nbc = СlassificationNB()
my_nbc.fit(x_train, y_train)

({'Pclass': [0.425, 2.0240641711229945, 0.7668005948125481],
  'sex_rank': [0.425, 0.7620320855614974, 0.18133918613629216],
  'Age': [0.425, 28.89596256684492, 208.96423797720837],
  'SibSp': [0.425, 0.5053475935828877, 0.5654794246332465],
  'Parch': [0.425, 0.5481283422459893, 0.8840472990362893],
  'Fare': [0.425, 49.99249144385027, 4639.73812686688],
  'embarked_s': [0.425, 0.6443850267379679, 0.22915296405387625],
  'embarked_c': [0.425, 0.2887700534759358, 0.20538190969144096],
  'embarked_q': [0.425, 0.06684491978609626, 0.0623766764848866]},
 {'Pclass': [0.575, 1.5849802371541502, 0.6261775687793905],
  'sex_rank': [0.575, 0.11857707509881422, 0.10451655235982439],
  'Age': [0.575, 30.38717391304348, 185.6875673096752],
  'SibSp': [0.575, 0.5237154150197628, 1.111097658141824],
  'Parch': [0.575, 0.391304347826087, 0.9180271524316892],
  'Fare': [0.575, 26.23407867660798, 1355.5376934950268],
  'embarked_s': [0.575, 0.7351778656126482, 0.19469137152587915],
  'embarked_c': [0.

In [19]:
y_pred = my_nbc.predict(x_test)
y_pred

0      -20.470958
1      -20.781695
2      -12.796859
3      -20.780570
4      -20.764496
          ...    
215    -19.890614
216    -16.453426
217    -22.983195
218   -134.856837
219    -13.078595
Length: 220, dtype: float64 0     -65.179040
1      -8.534264
2     -43.431478
3      -8.532410
4      -8.513647
         ...    
215    -8.318395
216   -47.381173
217    -9.087807
218   -98.316835
219   -44.679317
Length: 220, dtype: float64


0      1
1      0
2      1
3      0
4      0
      ..
215    0
216    1
217    0
218    0
219    1
Length: 220, dtype: int64

In [20]:
sklearn.metrics.accuracy_score(y_pred, y_test)

0.8045454545454546

Видим достаточно хороший score близкий к результату sklearn

### Полезные ссылки для этого классификатора

**Нелохие статьи для лучшего понимания**
- https://dzen.ru/a/YlrgU9faUy9WDu1l
- https://www.guru99.com/ru/naive-bayes-classifiers.html
- https://datascience.stackexchange.com/questions/27624/difference-between-bernoulli-and-multinomial-naive-bayes
- https://coderzcolumn.com/tutorials/machine-learning/scikit-learn-sklearn-naive-bayes
- https://datareview.info/article/6-prostyih-shagov-dlya-osvoeniya-naivnogo-bayesovskogo-algoritma-s-primerom-koda-na-python/

**Полезные ответы на stackoverflow**
- https://stackoverflow.com/questions/14254203/mixing-categorial-and-continuous-data-in-naive-bayes-classifier-using-scikit-lea
- https://stackoverflow.com/questions/38621053/how-can-i-use-sklearn-naive-bayes-with-multiple-categorical-features